In [ ]:
# Libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.types import StructType,StructField
import os
import numpy as np

In [ ]:
spark = SparkSession.builder.\
                    appName("spotify").\
                    config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1").\
                    config("spark.mongodb.input.uri", "mongodb+srv://**********").\
                    config("spark.mongodb.output.uri", "mongodb+srv://**********").\
                    config("spark.network.timeout", "7200s").\
                    config("spark.executor.heartbeatInterval", "1200s").\
                    config("spark.driver.maxResultSize", "24120").\
                    getOrCreate()

In [ ]:
# Connection info
database = '*****'
collection = 'songs_lyrics'
user_name = '*****'
password = '*****'
address = '**********.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

### Reading the main song dataframe

In [ ]:
df = spark.read.format('mongo').option("uri", connection_string).load()

In [ ]:
df_with_lyrics = df.where(df['lyrics'].isNotNull()).select('track_uri','lyrics', 'lyrics_words').distinct().cache()

In [ ]:
def get_me_tf_idf_scores(df_with_lyrics):
    """
    Input: Pass in a songs dataframe with lyrics 
    
    Output: dataframe with two columns: "track uri" and "word_and_tf_idf", which 
    contain lyric words and tf-idf scores in a list
    """
    
    hashingTF = HashingTF(inputCol="lyrics_words", outputCol="lyrical_features", numFeatures=16777216)
    featurized_data = hashingTF.transform(df_with_lyrics)
    idf = IDF(inputCol="lyrical_features", outputCol="lyrical_features_idf")
    idfModel = idf.fit(featurized_data)
    rescaledData = idfModel.transform(featurized_data)
    
    ### Finding hash of word
    ndf = df_with_lyrics.select('track_uri', explode('lyrics_words').alias('words')).\
                                            withColumn('lyrics_words', array('words'))

    hashudf = udf(lambda x: x.indices.tolist()[0], StringType())

    wordhash = hashingTF.transform(ndf).withColumn('word_hash', hashudf('lyrical_features'))

    ### Finding tfidf value of each word
    tfidfvalues = udf(lambda x: dict(zip(x.indices.tolist(),x.values.tolist())), MapType(StringType(), DoubleType()))

    wordtfidf = rescaledData.select('track_uri', explode(tfidfvalues('lyrical_features_idf'))).\
                             withColumnRenamed('key','word_hash').\
                             withColumnRenamed('value', 'word_tf_idf_score').distinct()

    ### Ranking tf idf scores
    w = Window.partitionBy('track_uri').orderBy(desc('word_tf_idf_score'))


    ### Joining with word hash value to get words                                                                                 
    final_df = wordhash.join(wordtfidf,['track_uri', 'word_hash'], 'left').\
                     withColumn('word_and_tf_idf', concat('word_tf_idf_score',lit(":"),'words')).distinct()

    ### Creating a list of word and tfidf score per song
    final = final_df.select('track_uri','word_and_tf_idf').\
                  groupBy('track_uri').\
                  agg(sort_array(collect_list('word_and_tf_idf'),asc=False).alias('tf_idf_and_word'))
    return final

In [ ]:
final = get_me_tf_idf_scores(df_with_lyrics)

#### Storing TF-IDF scores in word_tf_idf_score collection in mongo db

In [ ]:
database = '*****'
collection = 'word_tf_idf_score'
user_name = '*****'
password = '*****'
address = '**********.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
final.write.format("mongo").option("uri",connection_string).mode("append").save()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-548855102382580> in <module> 
 5 address = 'msds-697.a2nrn.mongodb.net' 
 6 connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}" 
 ----> 7 final . write . format ( "mongo" ) . option ( "uri" , connection_string ) . mode ( "append" ) . save ( ) 

 NameError : name 'final' is not defined

### Getting the song embedding trained by Chandrish

In [ ]:
database = '*****'
collection = 'song_vectors'
user_name = '*****'
password = '*****'
address = '**********.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
song_df = spark.read.format('mongo').option("uri", connection_string).load()

### Joining the songs dataframe with the song embeddings on track uri

In [ ]:
win = Window.partitionBy('pid','track_uri').orderBy('track_uri')

df = df.withColumn('num_occurences_in_playlist', row_number().over(win)) 

track_occurences = df.select('track_uri','pid','num_occurences_in_playlist').distinct()

song_embeddings_playlists = df.select('name','pid','track_uri','track_name','artist_name').distinct().\
                    join(song_df, 'track_uri', 'left').\
                    join(track_occurences, ['track_uri','pid'], 'left').cache()


In [ ]:
def id_sentiment(x):
    try :
        return (x[0], x[1])
    except:
        None

In [ ]:
temp = song_embeddings_playlists.select('name', 'values').rdd.map(lambda x: id_sentiment(x)).cache().groupByKey().\
                                       map(lambda x: (x[0], [float(x) for x in (np.mean(list(x[1]),axis = 0))])).\
                                       map(lambda x: (x[0],list(x[1])))


In [ ]:
struct1 = StructType([StructField('name',StringType(),True),StructField('playlist_embedding',ArrayType(FloatType()),True)])
playlist_avg_embedding = spark.createDataFrame(temp,struct1)

### Calculate Euclidean distance between the playlist embedding and song embedding

In [ ]:
def calculate_distance(arr_1, arr_2):
    # Calculate the distance between two lists.
    try:
        arr_1, arr_2 = np.array(arr_1), np.array(arr_2)
        return float(np.linalg.norm(arr_1 - arr_2))
    except:
        return -99999.99

distance = udf(lambda x, y: calculate_distance(x, y), FloatType())
    
song_embeddings_playlists =  song_embeddings_playlists.join(playlist_avg_embedding, ['name'],'left')


win_play = Window.partitionBy('name').orderBy('song_playlist_distance')


In [ ]:
song_embeddings_playlists = song_embeddings_playlists.withColumnRenamed('values', 'song_embedding')
                                    
song_embeddings_playlists = song_embeddings_playlists. withColumn('song_playlist_distance', distance(song_embeddings_playlists['song_embedding'],
                                    song_embeddings_playlists['playlist_embedding'])).withColumn('rank', rank().over(win_play)).cache()

In [ ]:
playlist_embeddings_final = song_embeddings_playlists.select('name','playlist_embedding').distinct()

In [ ]:
playlist_embeddings_final.show(10)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3164413804761074> in <module> 
 ----> 1 playlist_embeddings_final . show ( 10 ) 

 NameError : name 'playlist_embeddings_final' is not defined

### Saving playlist embedding in mongo db

In [ ]:
### Saving playlist embeddings
database = '*****'
collection = 'playlist_embeddings'
user_name = '*****'
password = '*****'
address = '**********.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
playlist_embeddings_final.write.format("mongo").option("uri",connection_string).mode("overwrite").save()

### Finding top 10 songs in the Rock playlist

In [ ]:
song_embeddings_playlists.where(song_embeddings_playlists['name'] == 'Rock').select('track_name','artist_name').orderBy('rank').distinct().show(truncate = False)

+----------------------------------+------------------------------------+
track_name |artist_name |
+----------------------------------+------------------------------------+
Charlie I Gotta Go |Black Star Riders |
The One I Love (Metal Mix) |Death Blossoms |
An Aggregation of Misfortune |Afgrund |
Hit The Wall |Galactic |
Hallucinagenda |Infanticide |
Three Lock Box - Live in St. Louis|Sammy Hagar |
Starlight - Live |Slash |
Godzilla |Blue Öyster Cult |
Лада Седан |Рекорд Оркестр |
When The Guns Come Out |Christ Bearer of Northstar, E-40, WC|
XXIII |Barbott |
No Resolve |Dirty Thrills |
Fallen |Stemm |
Solder Of Fortune |Toše Proeski |
Malaguegna Salerosa |De la Muerte |
Jump On My Ship |Here Come The Mummies |
Breakdown |Prospect Hill |
This Is for the Fallen |All Good Things |
Save Yourself |Follow You Home |
Celebration Song |Unwritten Law |
+----------------------------------+------------------------------------+
only showing top 20 rows

### Finding top 10 songs in the Country playlist

In [ ]:
song_embeddings_playlists.where(song_embeddings_playlists['name'] == 'Country').select('track_name','artist_name').orderBy('rank').distinct().show(truncate = False)

+-----------------------------------+--------------------------+
track_name |artist_name |
+-----------------------------------+--------------------------+
I Wanna Take a Little Ride With You|Top 40 Pop Hits |
Every Day's A Holiday |the JaneDear girls |
Caroline & Carolina |The Morrison Brothers Band|
Stray |Jason Michael Carroll |
It Takes A Man |Chris Young |
Letter to Me (Live) |Brad Paisley |
You Make It Look So Easy |Eric Church |
Only A Girl |Easton Corbin |
Night Of Our Lives |Rascal Flatts |
Live the Dream |Rachel Potter |
Night of Your Life |Kyle Setter |
That Girl Is a Cowboy |The Tribute Co. |
Ray Bans |JT Hodges |
Boy & A Girl Thing |Mo Pitney |
Those I've Loved |Eric Church |
Highway 20 Ride |Zac Brown Band |
Pretty Girls & Fishing Poles |Doug McCormick |
Tipsy |Jake Owen |
This I Gotta See |Jason Aldean |
Old Boots, New Dirt - Commentary |Jason Aldean |
+-----------------------------------+--------------------------+
only showing top 20 rows

### Finding top songs in the K Pop playlist

In [ ]:
song_embeddings_playlists.select('track_name', 'artist_name').where(song_embeddings_playlists['name'] == 'K Pop').orderBy('rank').distinct().show()

+------------------------+-----------------+
 track_name| artist_name|
+------------------------+-----------------+
 Cold Rain| 4Minute|
 Trouble (Prod. By...| Kriesha Chu|
 Lucifer| SHINee|
 Catch Me If You C...|Girls' Generation|
 Vibrato| Stellar|
 Give Love| AKDONG MUSICIAN|
 You Better Know| Red Velvet|
 BABY BABY| WINNER|
 Lies| BIGBANG|
 Everytime| Chenxpunch|
 HOLUP!| BOBBY|
 Boys and Girls (f...| Zico|
 Lucky Girl| Red Velvet|
 I don't need a man| miss A|
 STAY WITH ME - KR...| SOL|
 Tomorrow (feat. T...| Tablo|
 U & Me| Hyuna|
Replay -君は僕のevery...| SHINee|
 SHAKE IT| SISTAR|
 I LOVE YOU| 2NE1|
+------------------------+-----------------+
only showing top 20 rows

### Finding top songs in the Jazz playlist

In [ ]:
song_embeddings_playlists.select('track_name','artist_name').where(song_embeddings_playlists['name'] == 'Jazz standards').orderBy('rank').distinct().show()

+--------------------+--------------------+
 track_name| artist_name|
+--------------------+--------------------+
Someone To Watch ...| Jimmy Raney Trio|
 Well, You Needn't| Roy Ayers|
 Native Sense| Chick Corea|
 Bebop| Charlie Parker|
 Maple Leaf Rag| Scott Joplin|
 Lady Bird| Miles Davis|
 Groovin High| Charlie Parker|
Softly As In A Mo...|The Modern Jazz Q...|
 Wave|Antônio Carlos Jobim|
 Spain| Marian McPartland|
Boogie Woogie On ...| Earl Hines|
 All Blues|The George Benson...|
 How High The Moon| Charlie Parker|
 So What| Miles Davis|
Just Friends - Re...| Sonny Rollins|
 Tea for Two| Art Tatum|
 Blue Monk| Thelonious Monk|
Watermelon Man - ...| Herbie Hancock|
Someone to Watch ...| Jimmy Raney|
 Blues for Alice| Rahsaan Roland Kirk|
+--------------------+--------------------+
only showing top 20 rows